In [1]:
with open("./input.txt") as f:
    moon_location_strings = f.readlines()

In [2]:
import re

moon_location_tuples = []

for moon_location in moon_location_strings:
    location = re.findall("-?\d+", moon_location)
    moon_location_tuples.append(tuple(map(int, location)))

moon_location_tuples

[(5, -1, 5), (0, -14, 2), (16, 4, 0), (18, 1, 16)]

In [3]:
from typing import List, Tuple

class Moon:

    def __init__(self, position: Tuple[int, int, int]):
        self.position = list(position)
        self.velocity = [0, 0, 0]

    @property
    def x(self):
        return self.position[0]

    @property
    def y(self):
        return self.position[1]

    @property
    def z(self):
        return self.position[2]
        
    @property
    def vx(self):
        return self.velocity[0]

    @property
    def vy(self):
        return self.velocity[1]

    @property
    def vz(self):
        return self.velocity[2]

    def update_velocity(self, other_moons):
        """Updates moon's velocity based on other moon locations"""
        for other_moon in other_moons:
            for i in range(3):
                if self.position[i] < other_moon.position[i]:
                    self.velocity[i] += 1
                elif self.position[i] > other_moon.position[i]:
                    self.velocity[i] -= 1

    def update_position(self):
        """Updates moon's position based on its velocity"""
        for i in range(3):
            self.position[i] += self.velocity[i]

    def get_total_energy(self):
        """Returns product of absolute values of position and velocity"""
        potential_energy = sum(map(abs, self.position))
        kinetic_energy = sum(map(abs, self.velocity))
        return potential_energy * kinetic_energy

    def __str__(self):
        pos_x, pos_y, pos_z = self.position
        vel_x, vel_y, vel_z = self.velocity
        return f"pos=<x={pos_x:3}, y={pos_y:3}, z={pos_z:3}>, vel=<x={vel_x:3}, y={vel_y:3}, z={vel_z:3}>"

## part1

In [4]:
moons = [Moon(moon_location) for moon_location in moon_location_tuples]

for i in range(1000):
    for moon in moons:
        moon.update_velocity(moons)

    for moon in moons:
        moon.update_position()

f"part 1: total energy = {sum(moon.get_total_energy() for moon in moons)}"

'part 1: total energy = 7928'

## part2

The intituition here is that x, y, and z are cyclical and independent
of one another. We must simulate until we find the first time at which
x repeats itself, y repeats itself, and z repeats itself. Then, the
entire state repeats itself at the least common multiple of those.

In [5]:
import math

moons = [Moon(moon_location) for moon_location in moon_location_tuples]
init_x = [moon.x for moon in moons]
init_y = [moon.y for moon in moons]
init_z = [moon.z for moon in moons]
x_cycle, y_cycle, z_cycle = 0, 0, 0

step = 1
while x_cycle == 0 or y_cycle == 0 or z_cycle == 0:
    for moon in moons:
        moon.update_velocity(moons)
    
    for moon in moons:
        moon.update_position()
    
    cur_x = [moon.x for moon in moons]
    cur_y = [moon.y for moon in moons]
    cur_z = [moon.z for moon in moons]

    if x_cycle == 0 and all(moon.vx == 0 for moon in moons) and init_x == cur_x:
        x_cycle = step

    if y_cycle == 0 and all(moon.vy == 0 for moon in moons) and init_y == cur_y:
        y_cycle = step

    if z_cycle == 0 and all(moon.vz == 0 for moon in moons) and init_z == cur_z:
        z_cycle = step

    step += 1

f"part 2: repeats initial state at step: {math.lcm(x_cycle, y_cycle, z_cycle)}"

'part 2: repeats initial state at step: 518311327635164'